In [10]:
import json
import csv
from copy import deepcopy
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go
import numpy as np
import os
# pd.set_option('display.float_format', lambda x: '%.3f' % x)
# from IPython.core.display import display, HTML
# !display(HTML(“<style>.container { width:80% !important; }</style>“))

In [85]:
data = json.load(open('/Users/max.bade/Downloads/pancakeswap_coins.txt'))

def cross_join(left, right):
    new_rows = []
    for left_row in left:
        for right_row in right:
            temp_row = deepcopy(left_row)
            for key, value in right_row.items():
                temp_row[key] = value
            new_rows.append(deepcopy(temp_row))
    return new_rows


def flatten_list(data):
    for elem in data:
        if isinstance(elem, list):
            yield from flatten_list(elem)
        else:
            yield elem


def json_to_dataframe(data_in):
    def flatten_json(data, prev_heading=''):
        if isinstance(data, dict):
            rows = [{}]
            for key, value in data.items():
                rows = cross_join(rows, flatten_json(value, prev_heading + '.' + key))
        elif isinstance(data, list):
            rows = []
            for i in range(len(data)):
                [rows.append(elem) for elem in flatten_list(flatten_json(data[i], prev_heading))]
        else:
            rows = [{prev_heading[1:]: data}]
        return rows

    return pd.DataFrame(flatten_json(data_in))


if __name__ == '__main__':
    df = json_to_dataframe(data)
    df.columns = ['buy_symbol','sell_symbol','#_trades','trade_amount_$']
    df['num_trades_x_tradeamount'] = df['#_trades'] * df['trade_amount_$']
    df = df.sort_values(by='num_trades_x_tradeamount',ascending=False).reset_index()
    print(df)

     index buy_symbol sell_symbol  #_trades  trade_amount_$  \
0        0       BUSD        WBNB    238999   262498069.770   
1        1       WBNB        BUSD    237546   261352487.634   
2        4       Cake        WBNB    146410    57425499.867   
3        2       WBNB    SAFEMOON     94819    86607693.891   
4       11       WBNB       BUNNY    145229    36157925.904   
..     ...        ...         ...       ...             ...   
995    890        DIS        WBNB       107      200406.290   
996    885       WBNB        bLEO        91      204275.294   
997    991       BUSD          DX       108      161787.149   
998    935       WBNB     UNICORN        68      182357.295   
999    290       SAFU        WBNB         1     1288882.628   

     num_trades_x_tradeamount  
0          62736776176938.789  
1          62083238027443.891  
2           8407667435540.632  
3           8212054927042.438  
4           5251179421103.349  
..                        ...  
995              21

In [84]:
df['num_trades_x_tradeamount'] = df['#_trades'] * df['trade_amount_$']
df = df.sort_values(by='num_trades_x_tradeamount',ascending=False).reset_index()
df.head(20)

,index,buy_symbol,sell_symbol,#_trades,trade_amount_$,num_trades_x_tradeamount
0,0,BUSD,WBNB,238999,262498069.770,62736776176938.789
1,1,WBNB,BUSD,237546,261352487.634,62083238027443.891
2,4,Cake,WBNB,146410,57425499.867,8407667435540.632
3,2,WBNB,SAFEMOON,94819,86607693.891,8212054927042.438
4,11,WBNB,BUNNY,145229,36157925.904,5251179421103.349
5,7,USDT,WBNB,98457,42170315.574,4151962760491.640
6,8,WBNB,USDT,81792,42109778.263,3444242983647.673
7,3,SAFEMOON,WBNB,38980,78247163.353,3050074427514.440
8,5,WBNB,Cake,43750,56879590.642,2488482090597.602
9,10,WBNB,CUMMIES,39950,37223501.700,1487078892909.037


In [74]:
coins_to_remove = ['BUSD','WBNB','ETH','USDT','USDC','BTC','Cake']
df1 = df[~df.buy_symbol.isin(coins_to_remove)].head(100)
df2 = df[~df.sell_symbol.isin(coins_to_remove)].head(100)
df1.head()

,buy_symbol,sell_symbol,#_trades,trade_amount_$,num_trades_x_tradeamount
2,SAFEMOON,WBNB,41935,81162073.932,3403531570338.039
6,BUNNY,WBNB,26081,48737076.017,1271111679597.978
22,NFTART,WBNB,31992,20129674.865,643988558294.336
10,EPS,WBNB,12652,40471628.073,512047038377.303
15,BSCPAD,WBNB,11767,33939339.250,399364204956.553


In [82]:
fig = px.bar(df1
             ,x='buy_symbol'
             ,y='#_trades'
             ,color='buy_symbol'
#              ,line_shape='spline'
             ,template='plotly_white'
             ,color_discrete_sequence=px.colors.qualitative.Vivid
             ,title='<b>Top Buy Symbols by Trading Volume in Past Week on PancakeSwap')
fig.show()
fig = px.bar(df1
             ,x='buy_symbol'
             ,y='trade_amount_$'
             ,color='buy_symbol'
#              ,line_shape='spline'
             ,template='plotly_white'
             ,color_discrete_sequence=px.colors.qualitative.Prism
             ,title='<b>Top Buy Symbols by Trading Amount in Past Week on PancakeSwap')
fig.show()

fig = px.bar(df2
             ,x='sell_symbol'
             ,y='#_trades'
             ,color='sell_symbol'
#              ,line_shape='spline'
             ,template='plotly_white'
             ,color_discrete_sequence=px.colors.qualitative.Vivid
             ,title='<b>Top Sell Symbols by Trading Volume in Past Week on PancakeSwap')
fig.show()

fig = px.bar(df2
             ,x='sell_symbol'
             ,y='trade_amount_$'
             ,color='sell_symbol'
#              ,line_shape='spline'
             ,template='plotly_white'
             ,color_discrete_sequence=px.colors.qualitative.Prism
             ,title='<b>Top Sell Symbols by Trading Amount in Past Week on PancakeSwap')
fig.show()

# Trend Data

In [6]:
cd pancakeswap

/Users/max.bade/Downloads/pancakeswap


In [7]:
ls

4_1.txt   4_14.txt  4_19.txt  4_23.txt  4_28.txt  4_5.txt   5_1.txt   5_6.txt
4_10.txt  4_15.txt  4_2.txt   4_24.txt  4_29.txt  4_6.txt   5_2.txt   output/
4_11.txt  4_16.txt  4_20.txt  4_25.txt  4_3.txt   4_7.txt   5_3.txt
4_12.txt  4_17.txt  4_21.txt  4_26.txt  4_30.txt  4_8.txt   5_4.txt
4_13.txt  4_18.txt  4_22.txt  4_27.txt  4_4.txt   4_9.txt   5_5.txt


In [8]:
cake_4_1 = json.load(open('/Users/max.bade/Downloads/pancakeswap/4_1.txt'))
cake_4_2 = json.load(open('/Users/max.bade/Downloads/pancakeswap/4_2.txt'))
cake_4_3 = json.load(open('/Users/max.bade/Downloads/pancakeswap/4_3.txt'))
cake_4_4 = json.load(open('/Users/max.bade/Downloads/pancakeswap/4_4.txt'))
cake_4_5 = json.load(open('/Users/max.bade/Downloads/pancakeswap/4_5.txt'))
cake_4_6 = json.load(open('/Users/max.bade/Downloads/pancakeswap/4_6.txt'))
cake_4_7 = json.load(open('/Users/max.bade/Downloads/pancakeswap/4_7.txt'))
cake_4_8 = json.load(open('/Users/max.bade/Downloads/pancakeswap/4_8.txt'))
cake_4_9 = json.load(open('/Users/max.bade/Downloads/pancakeswap/4_9.txt'))
cake_4_10 = json.load(open('/Users/max.bade/Downloads/pancakeswap/4_10.txt'))
cake_4_11 = json.load(open('/Users/max.bade/Downloads/pancakeswap/4_11.txt'))
cake_4_12 = json.load(open('/Users/max.bade/Downloads/pancakeswap/4_12.txt'))
cake_4_13 = json.load(open('/Users/max.bade/Downloads/pancakeswap/4_13.txt'))
cake_4_14 = json.load(open('/Users/max.bade/Downloads/pancakeswap/4_14.txt'))
cake_4_15 = json.load(open('/Users/max.bade/Downloads/pancakeswap/4_15.txt'))
cake_4_16 = json.load(open('/Users/max.bade/Downloads/pancakeswap/4_16.txt'))
cake_4_17 = json.load(open('/Users/max.bade/Downloads/pancakeswap/4_17.txt'))
cake_4_18 = json.load(open('/Users/max.bade/Downloads/pancakeswap/4_18.txt'))
cake_4_19 = json.load(open('/Users/max.bade/Downloads/pancakeswap/4_19.txt'))
cake_4_20 = json.load(open('/Users/max.bade/Downloads/pancakeswap/4_20.txt'))
cake_4_21 = json.load(open('/Users/max.bade/Downloads/pancakeswap/4_21.txt'))
cake_4_22 = json.load(open('/Users/max.bade/Downloads/pancakeswap/4_22.txt'))
cake_4_23 = json.load(open('/Users/max.bade/Downloads/pancakeswap/4_23.txt'))
cake_4_24 = json.load(open('/Users/max.bade/Downloads/pancakeswap/4_24.txt'))
cake_4_25 = json.load(open('/Users/max.bade/Downloads/pancakeswap/4_25.txt'))
cake_4_26 = json.load(open('/Users/max.bade/Downloads/pancakeswap/4_26.txt'))
cake_4_27 = json.load(open('/Users/max.bade/Downloads/pancakeswap/4_27.txt'))
cake_4_28 = json.load(open('/Users/max.bade/Downloads/pancakeswap/4_28.txt'))
cake_4_29 = json.load(open('/Users/max.bade/Downloads/pancakeswap/4_29.txt'))
cake_4_30 = json.load(open('/Users/max.bade/Downloads/pancakeswap/4_30.txt'))
cake_5_1 = json.load(open('/Users/max.bade/Downloads/pancakeswap/5_1.txt'))
cake_5_2 = json.load(open('/Users/max.bade/Downloads/pancakeswap/5_2.txt'))
cake_5_3 = json.load(open('/Users/max.bade/Downloads/pancakeswap/5_3.txt'))
cake_5_4 = json.load(open('/Users/max.bade/Downloads/pancakeswap/5_4.txt'))
cake_5_5 = json.load(open('/Users/max.bade/Downloads/pancakeswap/5_5.txt'))
cake_5_6 = json.load(open('/Users/max.bade/Downloads/pancakeswap/5_6.txt'))

In [9]:
cakes = [cake_4_1, cake_4_2, cake_4_3, cake_4_4, cake_4_5, cake_4_6
         , cake_4_7, cake_4_8, cake_4_9, cake_4_10, cake_4_11, cake_4_12
         , cake_4_13, cake_4_14, cake_4_15, cake_4_16, cake_4_17, cake_4_18
         , cake_4_19, cake_4_20, cake_4_21, cake_4_22, cake_4_23, cake_4_24
         , cake_4_25, cake_4_26, cake_4_27, cake_4_28, cake_4_29, cake_4_30
         , cake_5_1, cake_5_2, cake_5_3, cake_5_4, cake_5_5, cake_5_6]

In [12]:
dirpath = '/Users/max.bade/Downloads/pancakeswap/'
output = '/Users/max.bade/Downloads/pancakeswap/output/pancakeswap.csv'
csvout_lst = []
files = [os.path.join(dirpath, fname) for fname in os.listdir(dirpath)]

# for filename in sorted(files):
for filename in cakes:
    
    def cross_join(left, right):
        new_rows = []
        for left_row in left:
            for right_row in right:
                temp_row = deepcopy(left_row)
                for key, value in right_row.items():
                    temp_row[key] = value
                new_rows.append(deepcopy(temp_row))
        return new_rows


    def flatten_list(data):
        for elem in data:
            if isinstance(elem, list):
                yield from flatten_list(elem)
            else:
                yield elem


    def json_to_dataframe(data_in):
        def flatten_json(data, prev_heading=''):
            if isinstance(data, dict):
                rows = [{}]
                for key, value in data.items():
                    rows = cross_join(rows, flatten_json(value, prev_heading + '.' + key))
            elif isinstance(data, list):
                rows = []
                for i in range(len(data)):
                    [rows.append(elem) for elem in flatten_list(flatten_json(data[i], prev_heading))]
            else:
                rows = [{prev_heading[1:]: data}]
            return rows

        return pd.DataFrame(flatten_json(data_in))

    if __name__ == '__main__':
        data = json_to_dataframe(filename)
        csvout_lst.append(data)
        
    pd.concat(csvout_lst).to_csv(output)

In [25]:
df = pd.read_csv(output,)
df.columns = ['index_duplicate','buy_symbol','sell_symbol','#_trades','trade_amount_$']
del df['index_duplicate']
df['num_trades_x_tradeamount'] = df['#_trades'] * df['trade_amount_$']
# df = df.sort_values(by='num_trades_x_tradeamount',ascending=False).reset_index()
# del df['index']
print(df.shape)

start = pd.to_datetime("4/1/2021")
df['trade_date'] = pd.Series(np.arange(len(df)) // 1000) \
              .apply(lambda x: pd.Timedelta(days=x)) + start

df['pair'] = df['buy_symbol'] + "/" + df['sell_symbol']
df['trade_date'] = pd.to_datetime(df['trade_date'])
df.sort_values(['pair','trade_date'], inplace = True, ascending=[True, True])
df['pct_ch'] = df.groupby(['pair'])['#_trades'].apply(lambda x: x/x.shift(1)-1).fillna(0)*100

df.iloc[0:20]

(36000, 5)


,buy_symbol,sell_symbol,#_trades,trade_amount_$,num_trades_x_tradeamount,trade_date,pair,pct_ch
2680,$BRUTO,WBNB,134,59729.137176,8.003704e+06,2021-04-03,$BRUTO/WBNB,0.000000
3723,$BRUTO,WBNB,156,52957.158039,8.261317e+06,2021-04-04,$BRUTO/WBNB,16.417910
6989,$BRUTO,WBNB,82,31140.805178,2.553546e+06,2021-04-07,$BRUTO/WBNB,-47.435897
757,$Cats,WBNB,94,34457.318973,3.238988e+06,2021-04-01,$Cats/WBNB,0.000000
1844,$Cats,WBNB,110,29314.920440,3.224641e+06,2021-04-02,$Cats/WBNB,17.021277
2956,$Cats,WBNB,70,20928.328322,1.464983e+06,2021-04-03,$Cats/WBNB,-36.363636
10992,$Cats,WBNB,134,37500.062739,5.025008e+06,2021-04-11,$Cats/WBNB,91.428571
11771,$Cats,WBNB,164,81902.982005,1.343209e+07,2021-04-12,$Cats/WBNB,22.388060
12787,$Cats,WBNB,147,64084.618594,9.420439e+06,2021-04-13,$Cats/WBNB,-10.365854
13655,$Cats,WBNB,195,85149.078063,1.660407e+07,2021-04-14,$Cats/WBNB,32.653061


In [33]:
sum_buy_symbol = pd.DataFrame(df.groupby('buy_symbol')['pct_ch'].agg('sum').reset_index().sort_values(by='pct_ch',ascending=False))
coins_to_keep = list(sum_buy_symbol.buy_symbol)
sum_buy_symbol.head(20)

,buy_symbol,pct_ch
1746,WBNB,341225.742706
948,MOONPIRATE,151550.659430
246,BUSD,30672.629607
702,HZN,13166.715975
667,HASH,13016.666667
19,100x,12034.740697
1902,upBNB,9799.128049
1358,SALT,6368.915244
1706,USDT,5662.918994
1353,SAFU,4122.003549


In [35]:
# df['#_trades'].value_counts()
# df[df['#_trades'] > 10000]['pair'].nunique()
coins_to_keep = ['MOONPIRATE','HZN','HASH','100X','upBNB','SALT','UNFI','DIS','MILK','BURGER','CODEX','MEL','PHX','WATER']
# coins_to_keep = ['BUSD','WBNB','ETH','USDT','USDC','BTC','Cake']
# df = df[~df.buy_symbol.isin(coins_to_remove)].head(100)
# df.pair.nunique()
df1 = df[df['buy_symbol'].isin(coins_to_keep)]

In [40]:
fig = px.bar(df1
             ,x='trade_date'
             ,y='pct_ch'
             ,color='pair'
             ,log_y=True
#              ,line_shape='spline'
             ,template='plotly_white'
             ,color_discrete_sequence=px.colors.qualitative.Pastel
             ,title='<b>Top Coin Pairs by Daily % Change on PancakeSwap')
fig.show()

# Test

In [ ]:
df['#_trades'] = df['#_trades'].astype('int64')
df.dtypes

In [ ]:
df['pair'] = df['pair'].fillna(0)
df = df[df['pair'].notna()]
df[df['pair'].isna()]

In [ ]:
df['pct_ch1'] = df.groupby(['pair','trade_date'])['#_trades'].pct_change()+1
df['pct_ch2'] = df.groupby(['trade_date','pair'])['#_trades'].apply(lambda x: x.pct_change()).to_numpy()
df['pct_ch3'] = df.groupby(['trade_date','pair'])['#_trades'].pct_change()
df['pct_ch4'] = df['#_trades'] / df.groupby(['trade_date','pair'])['#_trades'].shift(1)
df['pct_ch5'] = df.groupby(['trade_date','pair'])['#_trades'].apply(lambda x: x/x.shift(1))
df['pct_ch6'] = df.groupby(['trade_date','pair'])['#_trades'].apply(lambda x: x.pct_change())
df['pct_ch7'] = (df.groupby(['pair','trade_date'])['#_trades'].apply(pd.Series.pct_change) + 1)
df['pct_ch8'] = (df.groupby(['trade_date','pair'])['#_trades'].apply(lambda dfi : dfi.pct_change()) + 1)

In [ ]:
pd.concat(
    pd.read_csv(os.path.join(dirpath, fname), sep=':', index_col=0, header=None)
    for fname in sorted(os.listdir(dirpath))
).to_csv(output)

In [ ]:
start = pd.to_datetime("4/1/2021")
df['new_date'] = pd.Series(np.arange(len(df)) // 1000) \
              .apply(lambda x: pd.Timedelta(days=x)) + start

or

start = pd.to_datetime("4/1/2021")
df['d'] = start + (df.index // 1000 * pd.Timedelta(days=1))
print(df)

In [16]:
import json
import csv

def get_leaves(item, key=None):
    if isinstance(item, dict):
        leaves = []
        for i in item.keys():
            leaves.extend(get_leaves(item[i], i))
        return leaves
    elif isinstance(item, list):
        leaves = []
        for i in item:
            leaves.extend(get_leaves(i, key))
        return leaves
    else:
        return [(key, item)]


with open('pairs_json.json') as f_input, open('output.csv', 'w', newline='') as f_output:
    csv_output = csv.writer(f_output)
    write_header = True

    for entry in json.load(f_input):
        leaf_entries = sorted(get_leaves(entry))

        if write_header:
            csv_output.writerow([k for k, v in leaf_entries])
            write_header = False
            
        print([v for k,v in leaf_entries])
        csv_output.writerow([v for k, v in leaf_entries])

['ethereum']


In [18]:
def get_leaves(item, key=None):
    if isinstance(item, dict):
        leaves = {}
        for i in item.keys():
            leaves.update(get_leaves(item[i], i))
        return leaves
    elif isinstance(item, list):
        leaves = {}
        for i in item:
            leaves.update(get_leaves(i, key))
        return leaves
    else:
        return {key : item}


with open('pairs_json.json') as f_input:
    json_data = json.load(f_input)

# First parse all entries to get the complete fieldname list
fieldnames = set()

for entry in json_data:
    fieldnames.update(get_leaves(entry).keys())

with open('output.csv', 'w', newline='') as f_output:
    csv_output = csv.DictWriter(f_output, fieldnames=sorted(fieldnames))
    csv_output.writeheader()
    csv_output.writerows(get_leaves(entry) for entry in json_data)

In [37]:
import json
import pandas

data = json.load(open('/Users/max.bade/Downloads/pairs_json.txt'))

def cross_join(left, right):
    return left.assign(key=1).merge(right.assign(key=1), on='key', how='outer').drop('key', 1)

def json_to_dataframe(data_in):
    def to_frame(data, prev_key=None):
        if isinstance(data, dict):
            df = pandas.DataFrame()
            for key in data:
                df = cross_join(df, to_frame(data[key], prev_key + '.' + key))
        elif isinstance(data, list):
            df = pandas.DataFrame()
            for i in range(len(data)):
                df = pandas.concat([df, to_frame(data[i], prev_key)])
        else:
            df = pandas.DataFrame({prev_key[1:]: [data]})
        return df
    return to_frame(data_in)

if __name__ == '__main__':
    with open(data) as json_file:
        json_data = json.load(json_file)

    df = json_to_dataframe(json_data)
#     df.to_csv('output.csv', mode='w')

TypeError: expected str, bytes or os.PathLike object, not dict

In [36]:
data = json.load(open('/Users/max.bade/Downloads/pairs_json.json'))

JSONDecodeError: Extra data: line 1 column 3 (char 2)

In [27]:
# data = json.loads(raw)
data = json.load(open('/Users/max.bade/Downloads/pairs_json.json'))
data = pd.json_normalize(data)
# data.to_csv('output.csv')
# print(data.to_csv())
# df.to_csv('pancakswap_coins.csv', index = None, header=True)

In [28]:
data

,ethereum.dexTrades
0,"[{'buyCurrency': {'symbol': 'BUSD'}, 'sellCurr..."


In [39]:
df = pd.read_json('pancakeswap_coins.json')
df.head()

,ethereum
arguments,"[{'block': {'height': 7167593, 'timestamp': {'..."


In [41]:
df.to_csv('pancakswap_coins.csv', index = None, header=True)

In [8]:
!pwd

/Users/max.bade/Downloads


In [9]:
df = pd.read_json(x)
df.to_csv('test_json.csv')

In [1]:
import tkinter as tk
from tkinter import filedialog
from tkinter import messagebox
import pandas as pd

root= tk.Tk()

canvas1 = tk.Canvas(root
                    , width = 300
                    , height = 300
                    , bg = 'lightsteelblue2'
                    , relief = 'raised'
                   )
canvas1.pack()

label1 = tk.Label(root
                  , text='File Conversion Tool'
                  , bg = 'lightsteelblue2')
label1.config(font=('helvetica', 20))
canvas1.create_window(150, 60, window=label1)

def getJSON ():
    global read_file
    
    import_file_path = filedialog.askopenfilename()
    read_file = pd.read_json (import_file_path)
    
browseButton_JSON = tk.Button(text="      Import JSON File     "
                              , command=getJSON
                              , bg='green'
                              , fg='black'
                              , font=('helvetica', 12, 'bold')
                             )
canvas1.create_window(150, 130, window=browseButton_JSON)

def convertToCSV ():
    global read_file
    
    export_file_path = filedialog.asksaveasfilename(defaultextension='.csv')
    read_file.to_csv (export_file_path
                      , index = None
                      , header=True)

saveAsButton_CSV = tk.Button(text='Convert JSON to CSV'
                             , command=convertToCSV
                             , bg='green'
                             , fg='black'
                             , font=('helvetica', 12, 'bold'))
canvas1.create_window(150, 180, window=saveAsButton_CSV)

def exitApplication():
    MsgBox = tk.messagebox.askquestion ('Exit Application'
                                        ,'Are you sure you want to exit the application'
                                         ,icon = 'warning')
    if MsgBox == 'yes':
        root.destroy()
        
exitButton = tk.Button (root
                        , text='       Exit Application     '
                        , command=exitApplication
                        , bg='brown'
                        , fg='black'
                        , font=('helvetica', 12, 'bold'))
canvas1.create_window(150, 230, window=exitButton)

root.mainloop()

SystemExit: 0

/Users/max.bade/anaconda3/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3426: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)


In [9]:
df = pd.read_json(x)
df.head()

,ethereum
0,"{'arguments': [{'block': {'height': 7167593, '..."


In [21]:
data = json.load(open('/Users/max.bade/Downloads/pancakeswap_coins.json'))
jtopy=json.dumps(data) #json.dumps take a dictionary as input and returns a string as output.
x=json.loads(jtopy) # json.loads take a string as input and returns a dictionary as output.
# print(dict_json["shipments"])
x

{'ethereum': {'arguments': [{'block': {'height': 7167593,
     'timestamp': {'unixtime': 1620248019}},
    'argument': {'name': 'token1', 'type': 'address'},
    'reference': {'address': '0xd0e0204b1ad603df1cbeb10d5ab3b10a0b0a92bc',
     'smartContract': {'currency': {'name': '-'}}}},
   {'block': {'height': 7167593, 'timestamp': {'unixtime': 1620248019}},
    'argument': {'name': 'token0', 'type': 'address'},
    'reference': {'address': '0xbb4cdb9cbd36b01bd1cbaebf2de08d9173bc095c',
     'smartContract': {'currency': {'name': 'Wrapped BNB'}}}},
   {'block': {'height': 7167586, 'timestamp': {'unixtime': 1620247998}},
    'argument': {'name': 'token0', 'type': 'address'},
    'reference': {'address': '0x62c2455eb8b92ea8c7cb4bd0caeebca0cf846dea',
     'smartContract': {'currency': {'name': '-'}}}},
   {'block': {'height': 7167586, 'timestamp': {'unixtime': 1620247998}},
    'argument': {'name': 'token1', 'type': 'address'},
    'reference': {'address': '0xbb4cdb9cbd36b01bd1cbaebf2de08d91

In [33]:
# x = json.loads(x)
x = jtopy

f = csv.writer(open("pancakswap_coins.csv", "wb+"))

# Write CSV Header, If you dont need that, remove this line
# f.writerow(["pk", "model", "codename", "name", "content_type"])
f.writerow(["block", "height", "timestamp"
            , "name", "type", "address"
            , "currency", "name"])

# for x in x:
#     f.writerow([x["pk"],
#                 x["model"],
#                 x["fields"]["codename"],
#                 x["fields"]["name"],
#                 x["fields"]["content_type"]])

for x in x:
    f.writerow([x["ethereum"]["arguments"]["block"]["height"],
                x["ethereum"]["arguments"]["block"]["timestamp"]["unixtime"],
                x["ethereum"]["arguments"]["argument"]["name"],
                x["ethereum"]["arguments"]["argument"]["type"],
                x["ethereum"]["reference"]["address"],
                x["ethereum"]["reference"]["smartContract"]["currency"]["name"]
               ])

TypeError: a bytes-like object is required, not 'str'

In [37]:
x = json.load(open('/Users/max.bade/Downloads/pancakeswap_coins.json'))
with open(x) as json_file:
    data = json.dumps(json_file)

TypeError: expected str, bytes or os.PathLike object, not dict

In [34]:
# Python program to convert
# JSON file to CSV

import json
import csv
  
# Opening JSON file and loading the data
# into the variable data

x = json.load(open('/Users/max.bade/Downloads/pancakeswap_coins.json'))
with open(x) as json_file:
    data = json.load(json_file)
    
coins_data = data["ethereum"]["arguments"]
  
# now we will open a file for writing
data_file = open('data_file.csv', 'w')
  
# create the csv writer object
csv_writer = csv.writer(data_file)
  
# Counter variable used for writing 
# headers to the CSV file
count = 0
  
for coin in coins_data:
    if count == 0:
  
        # Writing headers of CSV file
        header = coin.keys()
        csv_writer.writerow(header)
        count += 1
  
    # Writing data of CSV file
    csv_writer.writerow(coin.values())

data_file.close()

TypeError: expected str, bytes or os.PathLike object, not dict